<a href="https://colab.research.google.com/github/ShuchengWang/CNN-3D-images-Tensorflow/blob/master/Run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## transefer the wrl format to a new folder
import os  
import shutil 
 

path = input('')
new_path = input('')
 
for root, dirs, files in os.walk(path):
    for i in range(len(files)):
        #print(files[i])
        if (files[i][-9:] == 'Depth.png'):
            file_path = root+'/'+files[i]  
            new_file_path = new_path+ '/'+ files[i]  
            shutil.copy(file_path,new_file_path)  

In [ ]:
import os
rootdir="/content/drive/MyDrive/Thesis_Project/DATA/Cald3r_train_data"
filelist=os.listdir(rootdir)
for file in filelist:
    if 'neutral_Depth.png' in file:
        del_file = rootdir + '/' + file 
        os.remove(del_file)
        print("已经删除：",del_file)


In [ ]:
import os,random,shutil
 
def moveFile(fileDir, tarDir):
    pathDir = os.listdir(fileDir)  
    filenumber = len(pathDir)
    rate = 0.2 
    picknumber = int(filenumber * rate)  
    sample = random.sample(pathDir, picknumber)  
    print(sample)
    print(len(sample))
    for name in sample:
        shutil.move(fileDir + name, tarDir + name)
 
 
if __name__ == '__main__':
    fileDir = "/content/drive/MyDrive/Thesis_Project/DATA/Cald3r_train_data/" 
    tarDir = '/content/drive/MyDrive/Thesis_Project/DATA/Cald3r_test_data/' 
    moveFile(fileDir, tarDir)

In [ ]:
import torch
import os
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
from torch import nn
from torch.nn import functional as F
import time

In [ ]:
!pip install trimesh

 Database

In [ ]:
import os
path = "/content/drive/MyDrive/Thesis_Project/DATA/BU_3DFE_train_data"
ori_files = os.listdir(path)

print(len(ori_files))

In [ ]:
import torch
from torch.utils.data import Dataset
import numpy as np
import open3d as o3d
import pandas as pd
class Dataset(Dataset):
    def __init__(self, file_name, file_dir):
        self.file_name = file_name
        self.file_dir = file_dir
    def __len__(self):
        return len(self.file_name)

    def __getitem__(self, idx):
      
        file_path = self.file_dir+'/'+self.file_name[idx]
        points = np.asarray(o3d.io.read_point_cloud(file_path).points,dtype='float32')
        n = np.random.choice(len(points), 1600, replace=False)
        point_data = points[n].T
        label = file_path[-14:-12]
        if label == 'HA':
          label = 0
        elif label == 'SA':
          label = 1
        elif label == 'SU':
          label = 2
        elif label == 'DI':
          label = 3
        elif label == 'FE':
          label = 4
        elif label == 'AN':
          label = 5
        return point_data, label

database = Dataset(ori_files,path)


In [ ]:
print(database[0][0])
print(database[0][1])
print(len(database))

[[  17.8223    50.0117   -36.7988  ...    8.23      48.6396    66.4781 ]
 [ -37.083    -39.5289   -81.3551  ... -110.949     48.4373    -3.90563]
 [  92.4608    79.2806    56.8212  ...   65.4938    73.3995    53.0297 ]]
4
1920


In [ ]:

import torch
from torch.utils.data import random_split
train_dataset, test_dataset = random_split(
    dataset=database,
    lengths=[2200,201],
    generator=torch.Generator().manual_seed(0)
)

In [ ]:
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

In [ ]:
print(len(train_dataset))


2200


K折交叉验证？？？？

Load Data

In [ ]:
from torch.utils.data import DataLoader
train_data = DataLoader(database, batch_size=50, shuffle=True)

In [ ]:
for i, (X, y) in enumerate(train_data):
  print('.....')
  print(X)
  print(y)

In [ ]:
next(iter(train_data))

In [ ]:
##print(f"Feature batch shape: {train_features.size()}")
for X, y in train_data:
  print(X)
  print(y)
  break

Accumulator

In [ ]:
class Accumulator: 
    
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
import torch
a = torch.tensor([2.33,43.4,33])
print(a.sum().item())

78.7300033569336


Set GPU

In [ ]:
def try_gpu(i=0):  
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')
  

Accuracy Function

In [ ]:
def accuracy(y_hat, y): 
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

Modol

In [ ]:
from torch.nn.modules.activation import ReLU
from torch.nn.modules.conv import Conv1d
import torch
from torch import nn

net = nn.Sequential(nn.Conv1d(3,64,1),nn.ReLU(),nn.Conv1d(64,64,1),nn.ReLU(),
                    nn.Conv1d(64,64,1),nn.ReLU(),nn.Conv1d(64,128,1),nn.ReLU(),
                    nn.Conv1d(128,1024,1),nn.ReLU(),nn.MaxPool1d(9600),
                    nn.Flatten(),
                    nn.Linear(1024, 512),nn.ReLU(),nn.Linear(512, 256),nn.ReLU(), 
                    nn.Dropout(0.7),nn.Linear(256, 6)) 



Train Function

In [ ]:
def train_function(net, train_iter, num_epochs, lr, device):
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv1d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    print('training on', device)
    net.to(device)
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    for epoch in range(num_epochs):
        metric = Accumulator(3)
        net.train()
        for i, (X, y) in enumerate(train_iter):
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l.sum(), accuracy(y_hat, y), y.numel())
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
        if (epoch+1)%10==0:
          print(f'num_epoch={epoch+1},loss={train_l},accuracy={train_acc}')     

In [ ]:
lr, num_epochs = 0.01, 3
train_function(net, train_data, num_epochs, lr, try_gpu())

Clear GPU memory

In [ ]:
!/opt/bin/nvidia-smi


In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

In [ ]:
import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()


Gen RAM Free: 52.6 GB  |     Proc size: 120.6 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total     16280MB


In [ ]:
!ps -aux|grep python

In [ ]:
!kill -9 5425	

In [ ]:
from posix import listdir
import os
a = listdir('/content/drive/MyDrive/Thesis_Project/DATA/UniformDownsample_Folder')
b = listdir('/content/drive/MyDrive/Thesis_Project/DATA/Translate_Folder')
c = listdir('/content/drive/MyDrive/Thesis_Project/DATA/Scale_Folder')
d = listdir('/content/drive/MyDrive/Thesis_Project/DATA/Rotate_Folder')
e = listdir('/content/drive/MyDrive/Thesis_Project/DATA/RandomDownsample_Folder')
f = listdir('/content/drive/MyDrive/Thesis_Project/DATA/Jitter_Folder')
g = listdir('/content/drive/MyDrive/Thesis_Project/DATA/ply_pcd')
print(len(a),len(b),len(c),len(d),len(e),len(f),len(g))

2400 7200 2400 7200 2400 2400 2400


In [ ]:
print(np.array(o3d.io.read_point_cloud('/content/drive/MyDrive/Thesis_Project/DATA/Scale_Folder/Scale_F0001_AN01WH_F3D.pcd').points))
print(o3d.io.read_point_cloud('/content/drive/MyDrive/Thesis_Project/DATA/Scale_Folder/Scale_F0001_AN01WH_F3D.pcd'))
print(np.array(o3d.io.read_point_cloud('/content/drive/MyDrive/Thesis_Project/DATA/ply_pcd/F0001_AN01WH_F3D.pcd').points))
print(o3d.io.read_point_cloud('/content/drive/MyDrive/Thesis_Project/DATA/ply_pcd/F0001_AN01WH_F3D.pcd'))
print(np.array(o3d.io.read_point_cloud('/content/drive/MyDrive/Thesis_Project/DATA/Scale_Folder/Scale_F0001_FE03WH_F3D.pcd').points))
print(o3d.io.read_point_cloud('/content/drive/MyDrive/Thesis_Project/DATA/Scale_Folder/Scale_F0001_FE03WH_F3D.pcd'))
print(np.array(o3d.io.read_point_cloud('/content/drive/MyDrive/Thesis_Project/DATA/ply_pcd/F0042_DI04AE_F3D.pcd').points))
print(o3d.io.read_point_cloud('/content/drive/MyDrive/Thesis_Project/DATA/ply_pcd/F0042_DI04AE_F3D.pcd'))

In [ ]:
! python /content/drive/MyDrive/Thesis_Project/DATA/Data_Helper.py

In [ ]:
a = []
path = '/content/drive/MyDrive/Thesis_Project/DATA/RandomDownsample_Folder'
h = listdir('/content/drive/MyDrive/Thesis_Project/DATA/RandomDownsample_Folder')

for i in h :
  ab_path = path +'/'+ i
  a.append(len(np.array(o3d.io.read_point_cloud(ab_path).points)))

print(a)

[4031, 3454, 3879, 4283, 3980, 3602, 3920, 4184, 4031, 3751, 4046, 4497, 4117, 3649, 4577, 3584, 3813, 3379, 3609, 3298, 3518, 3534, 3227, 3677, 3734, 3548, 3942, 3534, 3528, 3861, 3902, 3897, 3708, 3897, 3868, 4095, 3965, 4200, 3757, 3828, 3514, 3513, 3703, 3516, 3462, 3095, 3403, 3581, 3507, 3586, 3608, 3792, 3563, 3659, 3561, 3708, 3686, 3638, 3606, 3510, 3671, 3462, 3756, 4272, 3540, 3969, 4083, 3871, 3711, 3222, 4043, 3651, 3441, 3459, 3830, 3670, 3672, 4098, 3534, 3769, 3625, 3951, 3450, 3943, 3908, 3432, 3786, 3572, 3544, 3568, 3826, 3574, 3562, 3749, 3627, 3670, 3680, 3566, 3967, 3612, 3859, 3867, 3794, 3705, 3600, 3539, 3824, 3999, 4281, 4229, 3321, 3127, 3129, 3136, 3189, 3289, 3392, 3392, 3132, 3198, 3056, 3472, 2708, 3112, 2993, 2827, 3446, 3074, 3016, 2873, 3372, 3110, 3288, 3241, 4677, 4580, 4428, 4342, 4663, 4803, 4489, 3781, 3635, 4162, 3743, 3796, 3716, 4058, 3470, 3620, 4126, 3745, 3982, 3570, 3995, 4050, 4216, 3881, 3539, 3656, 3449, 3759, 3683, 3696, 3743, 3773, 374

In [ ]:
a = []
path = '/content/drive/MyDrive/Thesis_Project/DATA/RandomDownsample_Folder'
h = listdir('/content/drive/MyDrive/Thesis_Project/DATA/RandomDownsample_Folder')

for i in h :
  ab_path = path +'/'+ i
  a.append(len(np.array(o3d.io.read_point_cloud(ab_path).points)))

print(a)
max = a[0]
min = a[0]

for i in range(1, len(a)):
    
    if max < a[i]:
        max = a[i]

    if min > a[i]:
        min = a[i]
print(max)
print(min)

In [ ]:
a = []
path = '/content/drive/MyDrive/Thesis_Project/DATA/RandomDownsample_Folder'
h = listdir('/content/drive/MyDrive/Thesis_Project/DATA/RandomDownsample_Folder')

for i in h :
  ab_path = path +'/'+ i
  a.append(len(np.array(o3d.io.read_point_cloud(ab_path).points)))

print(a)
max = a[0]
min = a[0]

for i in range(1, len(a)):
    
    if max < a[i]:
        max = a[i]

    if min > a[i]:
        min = a[i]
print(max)
print(min)

[4031, 3454, 3879, 4283, 3980, 3602, 3920, 4184, 4031, 3751, 4046, 4497, 4117, 3649, 4577, 3584, 3813, 3379, 3609, 3298, 3518, 3534, 3227, 3677, 3734, 3548, 3942, 3534, 3528, 3861, 3902, 3897, 3708, 3897, 3868, 4095, 3965, 4200, 3757, 3828, 3514, 3513, 3703, 3516, 3462, 3095, 3403, 3581, 3507, 3586, 3608, 3792, 3563, 3659, 3561, 3708, 3686, 3638, 3606, 3510, 3671, 3462, 3756, 4272, 3540, 3969, 4083, 3871, 3711, 3222, 4043, 3651, 3441, 3459, 3830, 3670, 3672, 4098, 3534, 3769, 3625, 3951, 3450, 3943, 3908, 3432, 3786, 3572, 3544, 3568, 3826, 3574, 3562, 3749, 3627, 3670, 3680, 3566, 3967, 3612, 3859, 3867, 3794, 3705, 3600, 3539, 3824, 3999, 4281, 4229, 3321, 3127, 3129, 3136, 3189, 3289, 3392, 3392, 3132, 3198, 3056, 3472, 2708, 3112, 2993, 2827, 3446, 3074, 3016, 2873, 3372, 3110, 3288, 3241, 4677, 4580, 4428, 4342, 4663, 4803, 4489, 3781, 3635, 4162, 3743, 3796, 3716, 4058, 3470, 3620, 4126, 3745, 3982, 3570, 3995, 4050, 4216, 3881, 3539, 3656, 3449, 3759, 3683, 3696, 3743, 3773, 374

In [ ]:
a = []
path = '/content/drive/MyDrive/Thesis_Project/DATA/Jitter_Folder'
h = listdir('/content/drive/MyDrive/Thesis_Project/DATA/Jitter_Folder')

for i in h :
  ab_path = path +'/'+ i
  a.append(len(np.array(o3d.io.read_point_cloud(ab_path).points)))

print(a)
max = a[0]
min = a[0]

for i in range(1, len(a)):
    
    if max < a[i]:
        max = a[i]

    if min > a[i]:
        min = a[i]
print(max)
print(min)

[16124, 13818, 15516, 17134, 15920, 14410, 15680, 16738, 16126, 15006, 16184, 17990, 16470, 14598, 18308, 14338, 15254, 13518, 14438, 13194, 14074, 14136, 12908, 14708, 14936, 14194, 15770, 14138, 14114, 15444, 15610, 15590, 14832, 15588, 15474, 16380, 15860, 16800, 15030, 15312, 14056, 14054, 14812, 14064, 13848, 12380, 13612, 14324, 14028, 14344, 14434, 15168, 14252, 14636, 14246, 14834, 14744, 14552, 14426, 14042, 14686, 13850, 15024, 17090, 14160, 15876, 16334, 15486, 14844, 12890, 16172, 14606, 13764, 13838, 15322, 14680, 14690, 16392, 14136, 15076, 14502, 15806, 13802, 15772, 15632, 13730, 15146, 14290, 14178, 14274, 15306, 14298, 14248, 14996, 14508, 14680, 14722, 14264, 15870, 14448, 15436, 15470, 15176, 14822, 14400, 14158, 15296, 15998, 17126, 16918, 13284, 12508, 12518, 12544, 12756, 13156, 13568, 13568, 12528, 12792, 12224, 13890, 10832, 12450, 11972, 11310, 13784, 12296, 12066, 11492, 13490, 12442, 13152, 12966, 18710, 18322, 17714, 17368, 18654, 19212, 17958, 15124, 14542

In [ ]:
a = []
path = '/content/drive/MyDrive/Thesis_Project/DATA/Rotate_Folder'
h = listdir('/content/drive/MyDrive/Thesis_Project/DATA/Rotate_Folder')

for i in h :
  ab_path = path +'/'+ i
  a.append(len(np.array(o3d.io.read_point_cloud(ab_path).points)))

print(a)
max = a[0]
min = a[0]

for i in range(1, len(a)):
    
    if max < a[i]:
        max = a[i]

    if min > a[i]:
        min = a[i]
print(max)
print(min)

[6677, 7164, 7164, 7164, 7571, 7571, 7571, 7189, 7189, 7189, 6549, 6549, 6549, 7675, 7675, 7675, 7433, 7433, 7433, 6698, 6698, 6698, 6754, 6754, 6754, 6857, 6857, 6857, 7018, 7018, 7018, 6940, 6940, 6940, 6641, 6641, 6641, 6608, 6608, 6608, 7680, 7680, 7680, 6635, 6635, 6635, 6801, 6801, 6801, 6895, 6895, 6895, 7247, 7247, 7247, 6746, 6746, 6746, 7072, 7072, 7072, 6683, 6683, 6683, 6688, 6688, 6688, 7440, 7440, 7440, 6709, 6709, 6709, 7278, 7278, 7278, 7499, 7499, 7499, 7339, 7339, 7339, 6961, 6961, 6961, 6536, 6536, 6536, 7089, 7089, 7089, 7012, 7012, 7012, 6571, 6571, 6571, 6567, 6567, 6567, 7057, 7057, 7057, 7160, 7160, 7160, 7256, 7256, 7256, 7173, 7173, 7173, 7321, 7321, 7321, 7043, 7043, 7043, 7401, 7401, 7401, 7208, 7208, 7208, 6582, 6582, 6582, 6823, 6823, 6823, 6705, 6705, 6705, 7126, 7126, 7126, 6411, 6411, 6411, 6800, 6800, 6800, 7046, 7046, 7046, 7034, 7034, 7034, 6706, 6706, 6706, 6833, 6833, 6833, 7235, 7235, 7235, 7440, 7440, 7440, 6886, 6886, 6886, 7125, 7125, 7125, 667

In [ ]:
a = []
path = '/content/drive/MyDrive/Thesis_Project/DATA/UniformDownsample_Folder'
h = listdir('/content/drive/MyDrive/Thesis_Project/DATA/UniformDownsample_Folder')

for i in h :
  ab_path = path +'/'+ i
  a.append(len(np.array(o3d.io.read_point_cloud(ab_path).points)))

print(a)
max = a[0]
min = a[0]

for i in range(1, len(a)):
    
    if max < a[i]:
        max = a[i]

    if min > a[i]:
        min = a[i]
print(max)
print(min)

[4031, 3455, 3879, 4284, 3980, 3603, 3920, 4185, 4032, 3752, 4046, 4498, 4118, 3650, 4577, 3585, 3814, 3380, 3610, 3299, 3519, 3534, 3227, 3677, 3734, 3549, 3943, 3535, 3529, 3861, 3903, 3898, 3708, 3897, 3869, 4095, 3965, 4200, 3758, 3828, 3514, 3514, 3703, 3516, 3462, 3095, 3403, 3581, 3507, 3586, 3609, 3792, 3563, 3659, 3562, 3709, 3686, 3638, 3607, 3511, 3672, 3463, 3756, 4273, 3540, 3969, 4084, 3872, 3711, 3223, 4043, 3652, 3441, 3460, 3831, 3670, 3673, 4098, 3534, 3769, 3626, 3952, 3451, 3943, 3908, 3433, 3787, 3573, 3545, 3569, 3827, 3575, 3562, 3749, 3627, 3670, 3681, 3566, 3968, 3612, 3859, 3868, 3794, 3706, 3600, 3540, 3824, 4000, 4282, 4230, 3321, 3127, 3130, 3136, 3189, 3289, 3392, 3392, 3132, 3198, 3056, 3473, 2708, 3113, 2993, 2828, 3446, 3074, 3017, 2873, 3373, 3111, 3288, 3242, 4678, 4581, 4429, 4342, 4664, 4803, 4490, 3781, 3636, 4162, 3743, 3797, 3717, 4059, 3471, 3620, 4126, 3745, 3983, 3570, 3996, 4050, 4217, 3881, 3539, 3656, 3450, 3759, 3684, 3696, 3743, 3774, 374

In [ ]:
a = []
path = '/content/drive/MyDrive/Thesis_Project/DATA/Translate_Folder'
h = listdir('/content/drive/MyDrive/Thesis_Project/DATA/Translate_Folder')

for i in h :
  ab_path = path +'/'+ i
  a.append(len(np.array(o3d.io.read_point_cloud(ab_path).points)))

print(a)
max = a[0]
min = a[0]

for i in range(1, len(a)):
    
    if max < a[i]:
        max = a[i]

    if min > a[i]:
        min = a[i]
print(max)
print(min)

[6677, 7164, 7164, 7164, 7571, 7571, 7571, 7189, 7189, 7189, 6549, 6549, 6549, 7675, 7675, 7675, 7433, 7433, 7433, 6698, 6698, 6698, 6754, 6754, 6754, 6857, 6857, 6857, 7018, 7018, 7018, 6940, 6940, 6940, 6641, 6641, 6641, 6608, 6608, 6608, 7680, 7680, 7680, 6635, 6635, 6635, 6801, 6801, 6801, 6895, 6895, 6895, 7247, 7247, 7247, 6746, 6746, 6746, 7072, 7072, 7072, 6683, 6683, 6683, 6688, 6688, 6688, 7440, 7440, 7440, 6709, 6709, 6709, 7278, 7278, 7278, 7499, 7499, 7499, 7339, 7339, 7339, 6961, 6961, 6961, 6536, 6536, 6536, 7089, 7089, 7089, 7012, 7012, 7012, 6571, 6571, 6571, 6567, 6567, 6567, 7057, 7057, 7057, 7160, 7160, 7160, 7256, 7256, 7256, 7173, 7173, 7173, 7321, 7321, 7321, 7043, 7043, 7043, 7401, 7401, 7401, 7208, 7208, 7208, 6582, 6582, 6582, 6823, 6823, 6823, 6705, 6705, 6705, 7126, 7126, 7126, 6411, 6411, 6411, 6800, 6800, 6800, 7046, 7046, 7046, 7034, 7034, 7034, 6706, 6706, 6706, 6833, 6833, 6833, 7235, 7235, 7235, 7440, 7440, 7440, 6886, 6886, 6886, 7125, 7125, 7125, 667

In [ ]:
import random
a=[0,1,2,3,4,5,6,7,8,9]
b = random.sample(a, 5)
print(b)

[4, 5, 8, 2, 9]


In [ ]:
file_path = '/content/drive/MyDrive/Thesis_Project/DATA/Rotate_Folder/Rotate_X_F0001_AN03WH_F3D.pcd'

In [ ]:
points = np.array(pcd.points)
n = np.random.choice(len(points), 500, replace=False) 
pcd.points = o3d.utility.Vector3dVector(points[n])

In [ ]:

point_data = np.asarray(o3d.io.read_point_cloud(file_path).points,dtype='float32').T

In [ ]:
pcd = o3d.io.read_point_cloud(file_path)
points = np.asarray(o3d.io.read_point_cloud(file_path).points)
print(points)
len(points)
print(points[0])
n = np.random.choice(len(points), 1600, replace=False) 
print(n)
print(points[n].T)
print(np.asarray(o3d.utility.Vector3dVector(points[n])))
#pcd.points = o3d.utility.Vector3dVector(points[n])
#print(pcd.points)
#new_points = np.asarray(pcd.points)
#print(new_points)


In [ ]:
import os
a=os.listdir('/content/drive/MyDrive/Thesis_Project/DATA/Cald3r_test_data')
print(len(a))
b=os.listdir('/content/drive/MyDrive/Thesis_Project/DATA/Cald3r_train_data')
print(len(b))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive
!ls


In [ ]:
!pip install argparse
!pip install open3d

In [ ]:
! python /content/drive/MyDrive/Thesis_Project/main.py

In [ ]:
!pip install  matplotlib

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
!/opt/bin/nvidia-smi